In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf

In [ ]:
!wget http://ec2-3-141-45-250.us-east-2.compute.amazonaws.com/titanic.zip

--2021-06-07 20:39:11--  http://ec2-3-141-45-250.us-east-2.compute.amazonaws.com/titanic.zip
Resolving ec2-3-141-45-250.us-east-2.compute.amazonaws.com (ec2-3-141-45-250.us-east-2.compute.amazonaws.com)... 3.141.45.250
Connecting to ec2-3-141-45-250.us-east-2.compute.amazonaws.com (ec2-3-141-45-250.us-east-2.compute.amazonaws.com)|3.141.45.250|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K   169KB/s    in 0.2s    

2021-06-07 20:39:12 (169 KB/s) - ‘titanic.zip’ saved [34877/34877]



In [ ]:
!unzip titanic.zip; ls

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
gender_submission.csv  sample_data  test.csv  titanic.zip  train.csv


In [ ]:
gender_sub = pd.read_csv("gender_submission.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [ ]:
#test.tail()
#train.tail()
#gender_sub.tail()
#gender_sub.head()
print(test.axes)
print(train.axes)

[RangeIndex(start=0, stop=418, step=1), Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')]
[RangeIndex(start=0, stop=891, step=1), Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')]


In [ ]:
'''
train["Age"]=train["Age"].fillna(value=train["Age"].mean())
test["Age"]=test["Age"].fillna(value=test["Age"].mean())
test=test.dropna()
train=train.dropna()
print(len(test), len(train))
'''

'\ntrain["Age"]=train["Age"].fillna(value=train["Age"].mean())\ntest["Age"]=test["Age"].fillna(value=test["Age"].mean())\ntest=test.dropna()\ntrain=train.dropna()\nprint(len(test), len(train))\n'

In [ ]:
def convertCabin(s="0"):
  s=re.sub('\ ', '', s)
  s.lower()
  n=len(s)
  out=0
  for i in s:
    if i.isdigit() != True:
      out+=ord(i) * n * (10**(s.index(i)+1))
    else:
      out+=(10**(s.index(i)+1)+int(i))
  return int(out)
convertCabin()

10

In [ ]:
def convertEmbarked(s):
  return ord(s)
convertSex = lambda x : 1 if x=="male" else 0
def convertSurvived(x): 
  return 0.999 if int(x)==1 else 0

In [ ]:
max(train["Fare"])

512.3292

In [ ]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [ ]:
pd.unique(test["Embarked"])

array(['Q', 'S', 'C'], dtype=object)

In [ ]:
def removeChars(s):
    return int(re.sub('[\ a-zA-Z\/\.]', "", s))
    
def process(data, ans):
  copy=data.copy()
  copy["Age"]=copy["Age"].fillna(value=copy["Age"].mean())
  copy=copy.dropna()
  print(len(copy))
  copy=copy.drop(labels="Name", axis=1)
  copy["Ticket"]=copy["Ticket"].apply(removeChars)
  copy["Embarked"]=copy["Embarked"].apply(convertEmbarked)
  copy["Cabin"]=copy["Cabin"].apply(convertCabin)
  copy["Sex"]=copy["Sex"].apply(convertSex)
  copy=copy.join(gender_sub.set_index("PassengerId"), on="PassengerId",rsuffix="Right")
  #copy["Survived"]=copy["Survived"].apply(convertSurvived)
  print(np.max(copy['Embarked']))
  copy=copy.drop(labels="PassengerId", axis=1)
  if "SurvivedRight" in copy.axes[-1]:
    copy=copy.drop(labels="SurvivedRight", axis=1)
  #print(copy)
  copy=np.asarray(copy)
  for c in range(copy.shape[1]):
    copy[:,c] /= np.max(copy[:, c])+10**-9
    #copy[] = copy[c] / max
  return copy
#np.max(process(train, gender_sub)["Ticket"])
#pd.unique(process(train, gender_sub)["Cabin"])
process(train, gender_sub)[0]

202
83


array([9.99999999e-01, 3.33333333e-01, 0.00000000e+00, 4.75000000e-01,
       3.33333333e-01, 0.00000000e+00, 4.48844161e-02, 1.39135735e-01,
       2.40040043e-06, 8.07228916e-01])

In [ ]:
train_data = process(train, gender_sub)
test_data = process(test, gender_sub)
print(train_data.shape)
print(test_data.shape)

202
91
(202, 10)
(91, 10)


In [ ]:
train_data[0]

array([9.99999999e-01, 3.33333333e-01, 0.00000000e+00, 4.75000000e-01,
       3.33333333e-01, 0.00000000e+00, 4.48844161e-02, 1.39135735e-01,
       2.40040043e-06, 8.07228916e-01])

In [ ]:
test_data[0]

array([3.33333333e-01, 0.00000000e+00, 3.02631579e-01, 3.33333333e-01,
       0.00000000e+00, 6.09786224e-02, 1.60573904e-01, 2.37426735e-06,
       1.00000000e+00])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(9),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data[:,1:], train_data[:,0], epochs=10)

Epoch 1/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6716 - accuracy: 0.6139
Epoch 2/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6042 - accuracy: 0.6634
Epoch 3/10
7/7 [==============================] - 0s 4ms/step - loss: 0.5487 - accuracy: 0.6782
Epoch 4/10
7/7 [==============================] - 0s 6ms/step - loss: 0.5025 - accuracy: 0.7475
Epoch 5/10
7/7 [==============================] - 0s 5ms/step - loss: 0.4796 - accuracy: 0.7376
Epoch 6/10
7/7 [==============================] - 0s 3ms/step - loss: 0.4750 - accuracy: 0.7475
Epoch 7/10
7/7 [==============================] - 0s 3ms/step - loss: 0.4720 - accuracy: 0.7426
Epoch 8/10
7/7 [==============================] - 0s 3ms/step - loss: 0.4634 - accuracy: 0.7426
Epoch 9/10
7/7 [==============================] - 0s 3ms/step - loss: 0.4610 - accuracy: 0.7376
Epoch 10/10
7/7 [==============================] - 0s 4ms/step - loss: 0.4644 - accuracy: 0.7376


In [ ]:
print(train_data.shape)
print(test_data.shape)

(202, 10)
(91, 10)


In [ ]:
test_loss, test_acc = model.evaluate(test_data[:,:-1], test_data[:,-1], verbose=2)

print('\nTest accuracy:', test_acc)

3/3 - 0s - loss: 0.2802 - accuracy: 1.0000

Test accuracy: 1.0


In [ ]:
for i in range(10, 20):
  print(model.predict(test_data[i,:-1].reshape(-1, 9)))
  print(test_data[i,-1:])

[[ 0.14991227 -0.03200021]]
[0.]
[[-1.3639569  0.9386918]]
[1.]
[[ 0.5561538  -0.50033444]]
[0.]
[[-1.0537497  0.6678504]]
[1.]
[[ 0.6828544  -0.34082705]]
[0.]
[[ 0.36222032 -0.1864124 ]]
[0.]
[[-0.57070357  0.4698849 ]]
[1.]
[[ 0.37290177 -0.19900821]]
[0.]
[[-0.767002   0.4600895]]
[1.]
[[ 0.58067876 -0.3473177 ]]
[0.]


In [ ]:
test_data[-1,-1:]
model.predict(test_data[i,:-1].reshape(-1, 9))[0, 1]

-0.3473177

In [ ]:
model.save("./MyDrive/ColabN*/kaggle/model")

INFO:tensorflow:Assets written to: ./MyDrive/ColabN*/kaggle/model/assets


In [ ]:
!ls MyDrive/ColabN*/kaggle

Titanic.ipynb


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
drive.flush_and_unmount()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install django

     |████████████████████████████████| 7.9MB 2.9MB/s 


In [ ]:
!ls /datalab/
#!django-admin startproject mysite

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
run.sh	web


In [ ]:
tf.keras.models.load_model("ec2-3-141-45-250.us-east-2.compute.amazonaws.com/model-20210607t073640z-001.zip")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-214-dd45e4d90ca4>", line 1, in <module>
    tf.keras.models.load_model("ec2-3-141-45-250.us-east-2.compute.amazonaws.com/model-20210607t073640z-001.zip")
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/saving/save.py", line 200, in load_model
    (isinstance(filepath, h5py.File) or h5py.is_hdf5(filepath))):
  File "/usr/local/lib/python3.7/dist-packages/h5py/_hl/base.py", line 34, in is_hdf5
    fname = os.path.abspath(fspath(fname))
  File "/usr/lib/python3.7/posixpath.py", line 383, in abspath
    cwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshel

OSError: ignored